In [7]:
import os
import shutil
import time
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import logging
import re

logging.basicConfig(level=logging.INFO)

# Define constants
SOURCE_FOLDER = r'R:\RawData\Massmart Data'

PARAMETERS = [
  {'target_dir': r'R:\RawData\Masscash\\Wholesale\ToUpload', 'keyword': 'MassCashWholesale'},
  {'target_dir': r'R:\RawData\Masscash\Retail\ToUpload',     'keyword': 'MassCashRetail'},
  {'target_dir': r'R:\RawData\Makro Updated\\ToUpload',      'keyword': 'Makro'},
  {'target_dir': r'R:\RawData\Game\ToUpload',                'keyword': 'MassDiscounters'},
  {'target_dir': r'R:\RawData\KitKat\ToUpload',              'keyword': 'KitKat'},
  {'target_dir': r'R:\RawData\Massbuild\ToUpload',           'keyword': 'Massbuild'}
]

EXCLUDED_FILES_REGEX = r'^(exportsummary|stock|test).*'

MAX_ALLOWED_TIME_LAPSE = 60 * 5  # Five minutes

class FileHandler(FileSystemEventHandler):
  def __init__(self, source_dir, parameters):
    self.source_dir = source_dir
    self.parameters = parameters

  def on_created(self, event):
    for params in self.parameters:
      if params['keyword'] in event.src_path:
        move_files_based_on_keyword(self.source_dir, params['target_dir'], params['keyword'])

def move_files_based_on_keyword(source_dir, target_dir, keyword):
  # Get a list of files in the source directory that match the keyword and are not excluded
  files_to_move = [file for file in os.listdir(source_dir) 
                    if keyword in file 
                    and not any(re.match(EXCLUDED_FILES_REGEX, file.lower(), re.IGNORECASE))
                    and file.endswith('.txt') 
                  ]
  
  if not files_to_move:
    logging.info(f"No files with the keyword '{keyword}' found in {source_dir}. Skipping move operation.")
    return

  # Move each file to the target directory
  for file in files_to_move:
    source = os.path.join(source_dir, file)
    destination = os.path.join(target_dir, file)
    shutil.move(source, destination)
    logging.info(f"Moved {file} to {destination}")

def monitor_files(source_folder, parameters, max_inactive_time):
  for params in parameters:
    move_files_based_on_keyword(source_folder, params['target_dir'], params['keyword'])

  # Create a file handler and observer to monitor the source folder
  event_handler = FileHandler(source_folder, parameters)
  observer = Observer()
  observer.schedule(event_handler, source_folder, recursive=True)
  observer.start()

  try:
    logging.info(f"Monitoring {source_folder} for new files...")
    start_time = time.time()
    while True:
      if time.time() - start_time >= max_inactive_time:
        logging.info(f"No new files detected for the past {int(max_inactive_time/60)} minutes. Terminating script.")
        observer.stop()
        break

      time.sleep(10)
  except KeyboardInterrupt:
    observer.stop()
  observer.join()
  
if __name__ == "__main__":
  monitor_files(SOURCE_FOLDER, PARAMETERS, MAX_ALLOWED_TIME_LAPSE)

TypeError: 'NoneType' object is not iterable